In [1]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import re

# URL 및 요청 헤더 설정
# User-Agent는 서버에서 클라이언트가 어떤 소프트웨어를 사용하는지 파악하기 위해 User_Agent값을 확인한다함.
# "Mozilla/5.0"은 대표적인 웹 브라우저에서 많이 사용되는 문자열로, 브라우저에서 보내는 요청처럼 보이게 한다고함.
url = 'https://droneportal.or.kr/subList/22000000153'
headers = {'User-Agent': 'Mozilla/5.0'}
req = urllib.request.Request(url, headers=headers)

# 웹 페이지 요청 및 HTML 데이터 읽기
try:
    with urllib.request.urlopen(req) as response:
        html_data = response.read().decode('utf-8')
except Exception as e:
    print("웹 페이지 요청 실패:", e)
    exit()

# BeautifulSoup을 이용한 HTML 파싱
soup = BeautifulSoup(html_data, 'html.parser')

# "UA" 문자열이 포함된 테이블 찾기
tables = soup.find_all('table')
target_table = None
for table in tables:
    if "UA" in table.get_text():
        target_table = table
        break

if target_table is None:
    print("대상 테이블을 찾을 수 없습니다.")
    exit()

# 테이블 내 모든 행 추출
rows = target_table.find_all("tr")
data = []

# 각 행에서 셀 텍스트 추출 및 필터링
for row in rows:
    cells = row.find_all(["td", "th"])
    cell_texts = [cell.get_text(strip=True) for cell in cells]
    # 4열(구역 코드, 위치, 수평범위, 수직범위)이 부족하면 None 채움
    while len(cell_texts) < 4:
        cell_texts.append(None)
    # 첫 번째 셀에 "*"나 공백이 있더라도 "UA"와 숫자가 포함되면 데이터로 판단
    if cell_texts and re.search(r'^[\*\s]*UA\s*\d+', cell_texts[0]):
        data.append(cell_texts)

# DataFrame 생성 및 컬럼명 지정
columns = ["구역 코드", "위치", "수평범위", "수직범위"]
df = pd.DataFrame(data, columns=columns)

# -------------------------------------------
# Circle 데이터와 NE 데이터 추출 함수 정의
# -------------------------------------------
def extract_circle_data(text):
    """
    예시 패턴:
    "Circle with radius of 1.8 km (1.0 NM) centered on 354421N 1270027E"
    또는 "Circle with radius of 50m(0.03NM) centered on 351318N 1285142E"
    """
    match = re.search(r'Circle with radius of ([\d.]+)\s*(km|m).*?centered on\s*([\d]+)N\s*([\d]+)E', text)
    if match:
        radius = float(match.group(1))
        unit = match.group(2)
        if unit == "m":
            radius = radius / 1000  # m 단위를 km로 변환
        lat = float(match.group(3)) / 10000
        lon = float(match.group(4)) / 10000
        return radius, lat, lon
    return None, None, None

def extract_ne_data(text):
    """
    NE 데이터 예시:
    "373010N 1272300E - 373010N 1273200E - 372700N 1273200E - 372700N 1272300E - to point of origin"
    """
    coords = re.findall(r'(\d{6,7})N\s*([\d]{6,7})E', text)
    result = []
    for lat_str, lon_str in coords:
        lat = float(lat_str) / 10000
        lon = float(lon_str) / 10000
        result.append((lat, lon))
    return result if result else None

# -------------------------------------------
# DataFrame에 새로운 컬럼 추가: 반경, 위도, 경도, NE
# -------------------------------------------
df['반경'] = None
df['위도'] = None
df['경도'] = None
df['NE'] = None

# 각 행에 대해 "수평범위" 값에 따라 Circle 데이터 또는 NE 데이터를 추출
for idx, row in df.iterrows():
    range_text = row['수평범위']
    if range_text:
        if "Circle" in range_text:
            radius, lat, lon = extract_circle_data(range_text)
            df.at[idx, '반경'] = radius
            df.at[idx, '위도'] = lat
            df.at[idx, '경도'] = lon
        else:
            ne_coords = extract_ne_data(range_text)
            df.at[idx, 'NE'] = ne_coords

# -------------------------------------------
# Circle 테이블과 NE 테이블로 분리
# -------------------------------------------
# Circle 테이블: "Circle" 데이터가 있는 행만 선택 (반경 값이 존재하는 경우)
circle_table = df[df['반경'].notnull()][["구역 코드", "위치", "반경", "위도", "경도"]].reset_index(drop=True)

# NE 테이블: NE 데이터가 존재하는 행만 선택
ne_table = df[df['NE'].notnull()][["구역 코드", "위치", "NE"]].reset_index(drop=True)

# -------------------------------------------
# 결과 출력
# -------------------------------------------
print("=== Circle Table ===")
print(circle_table)
print("\n=== NE Table ===")
print(ne_table)


=== Circle Table ===
      구역 코드   위치    반경       위도        경도
0      UA 2  구성산   1.8  35.4421  127.0027
1      UA 3   약산   0.7  35.4421  128.2502
2      UA 4  봉화산   4.0  35.3731  129.0532
3      UA 5  덕두산   4.5  35.2441  127.3157
4      UA 6   금산   2.1  34.4411  127.5852
5      UA 7   홍산   1.2  35.4941  127.0452
6     UA 10   고창   4.0  35.2311  126.4353
7     UA 21  방장산   3.0  35.2658  126.4417
8     UA 24   구좌   2.8  33.2841  126.4922
9     UA 27  미악산   1.2    33.18  126.3316
10    UA 28  서운산   2.0   36.555  127.1659
11    UA 29   오천   2.0  36.5711  127.1716
12    UA 30   북좌   2.0  37.0242   127.194
13  * UA 32   퇴촌   0.3    37.28  127.1809
14  * UA 40   고령  0.08  35.5034  128.2639
15  **UA 42   광주  0.05  35.1318  128.5142

=== NE Table ===
      구역 코드   위치                                                 NE
0      UA 9   양평  [(37.301, 127.23), (37.301, 127.32), (37.27, 1...
1     UA 14   공주  [(36.3038, 127.0033), (36.3002, 127.0713), (36...
2     UA 19  시화호  [(37.1751, 126.4215), (37

In [3]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import re

# URL 및 요청 헤더 설정 - chat gpt 도움 받음
url = 'https://droneportal.or.kr/subList/22000000153'
headers = {'User-Agent': 'Mozilla/5.0'}
req = urllib.request.Request(url, headers=headers)

# 웹 페이지 요청 및 HTML 데이터 읽기
try:
    with urllib.request.urlopen(req) as response:
        html_data = response.read().decode('utf-8')
except Exception as e:
    print("웹 페이지 요청 실패:", e)
    exit()

# BeautifulSoup을 이용한 HTML 파싱
soup = BeautifulSoup(html_data, 'html.parser')

# "UA" 문자열이 포함된 테이블 찾기
tables = soup.find_all('table')
target_table = None
for table in tables:
    if "UA" in table.get_text():
        target_table = table
        break

if target_table is None:
    print("대상 테이블을 찾을 수 없습니다.")
    exit()

# 테이블 내 모든 행 추출
rows = target_table.find_all("tr")
data = []
for row in rows:
    cells = row.find_all(["td", "th"])
    cell_texts = [cell.get_text(strip=True) for cell in cells]
    while len(cell_texts) < 4:
        cell_texts.append(None)
    # 첫번째 셀에 "*"나 공백이 있더라도 "UA"와 숫자가 있으면 데이터로 판단
    if cell_texts and re.search(r'^[\*\s]*UA\s*\d+', cell_texts[0]):
        data.append(cell_texts)

# DataFrame 생성
columns = ["구역 코드", "위치", "수평범위", "수직범위"]
df = pd.DataFrame(data, columns=columns)

# Circle 데이터 추출 함수 (예: "Circle with radius of 1.8 km (1.0 NM) centered on 354421N 1270027E")
def extract_circle_data(text):
    match = re.search(r'Circle with radius of ([\d.]+)\s*(km|m).*?centered on\s*([\d]+)N\s*([\d]+)E', text)
    if match:
        radius = float(match.group(1))
        unit = match.group(2)
        if unit == "m":
            radius = radius / 1000  # m를 km로 변환
        lat = float(match.group(3)) / 10000
        lon = float(match.group(4)) / 10000
        return radius, lat, lon
    return None, None, None

# 새로운 컬럼 추가: 반경, 위도, 경도
df['반경'] = None
df['위도'] = None
df['경도'] = None

# "수평범위"에 "Circle"이 포함된 경우 데이터 추출
for idx, row in df.iterrows():
    range_text = row['수평범위']
    if range_text and "Circle" in range_text:
        radius, lat, lon = extract_circle_data(range_text)
        df.at[idx, '반경'] = radius
        df.at[idx, '위도'] = lat
        df.at[idx, '경도'] = lon

# Circle 테이블 생성 (반경 데이터가 존재하는 행)
circle_table = df[df['반경'].notnull()][["구역 코드", "위치", "반경", "위도", "경도"]].reset_index(drop=True)

print("=== Circle Table ===")
print(circle_table)


=== Circle Table ===
      구역 코드   위치    반경       위도        경도
0      UA 2  구성산   1.8  35.4421  127.0027
1      UA 3   약산   0.7  35.4421  128.2502
2      UA 4  봉화산   4.0  35.3731  129.0532
3      UA 5  덕두산   4.5  35.2441  127.3157
4      UA 6   금산   2.1  34.4411  127.5852
5      UA 7   홍산   1.2  35.4941  127.0452
6     UA 10   고창   4.0  35.2311  126.4353
7     UA 21  방장산   3.0  35.2658  126.4417
8     UA 24   구좌   2.8  33.2841  126.4922
9     UA 27  미악산   1.2    33.18  126.3316
10    UA 28  서운산   2.0   36.555  127.1659
11    UA 29   오천   2.0  36.5711  127.1716
12    UA 30   북좌   2.0  37.0242   127.194
13  * UA 32   퇴촌   0.3    37.28  127.1809
14  * UA 40   고령  0.08  35.5034  128.2639
15  **UA 42   광주  0.05  35.1318  128.5142


In [5]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import re

# URL 및 요청 헤더 설정
url = 'https://droneportal.or.kr/subList/22000000153'
headers = {'User-Agent': 'Mozilla/5.0'}
req = urllib.request.Request(url, headers=headers)

# 웹 페이지 요청 및 HTML 데이터 읽기
try:
    with urllib.request.urlopen(req) as response:
        html_data = response.read().decode('utf-8')
except Exception as e:
    print("웹 페이지 요청 실패:", e)
    exit()

# BeautifulSoup을 이용한 HTML 파싱
soup = BeautifulSoup(html_data, 'html.parser')

# "UA" 문자열이 포함된 테이블 찾기
tables = soup.find_all('table')
target_table = None
for table in tables:
    if "UA" in table.get_text():
        target_table = table
        break

if target_table is None:
    print("대상 테이블을 찾을 수 없습니다.")
    exit()

# 테이블 내 모든 행 추출
rows = target_table.find_all("tr")
data = []
for row in rows:
    cells = row.find_all(["td", "th"])
    cell_texts = [cell.get_text(strip=True) for cell in cells]
    while len(cell_texts) < 4:
        cell_texts.append(None)
    # 첫번째 셀에 "*"나 공백이 있더라도 "UA"와 숫자가 있으면 데이터로 판단
    if cell_texts and re.search(r'^[\*\s]*UA\s*\d+', cell_texts[0]):
        data.append(cell_texts)

# DataFrame 생성
columns = ["구역 코드", "위치", "수평범위", "수직범위"]
df = pd.DataFrame(data, columns=columns)

# NE 데이터 추출 함수 (예: "373010N 1272300E - 373010N 1273200E - ...")
def extract_ne_data(text):
    coords = re.findall(r'(\d{6,7})N\s*([\d]{6,7})E', text)
    result = []
    for lat_str, lon_str in coords:
        lat = float(lat_str) / 10000
        lon = float(lon_str) / 10000
        result.append((lat, lon))
    return result if result else None

# 새로운 컬럼 추가: NE
df['NE'] = None

# "수평범위"에 "Circle"이 포함되지 않은 경우 NE 데이터 추출
for idx, row in df.iterrows():
    range_text = row['수평범위']
    if range_text and "Circle" not in range_text:
        ne_coords = extract_ne_data(range_text)
        df.at[idx, 'NE'] = ne_coords

# NE 테이블 생성 (NE 데이터가 존재하는 행)
ne_table = df[df['NE'].notnull()][["구역 코드", "위치", "NE"]].reset_index(drop=True)

print("=== NE Table ===")
print(ne_table)


=== NE Table ===
      구역 코드   위치                                                 NE
0      UA 9   양평  [(37.301, 127.23), (37.301, 127.32), (37.27, 1...
1     UA 14   공주  [(36.3038, 127.0033), (36.3002, 127.0713), (36...
2     UA 19  시화호  [(37.1751, 126.4215), (37.1724, 126.5), (37.14...
3     UA 25   하동  [(35.0147, 127.4325), (35.0145, 127.4741), (34...
4     UA 26  장암산  [(37.2338, 128.2419), (37.241, 128.281), (37.2...
5   * UA 31   청라  [(37.3354, 126.373), (37.34, 126.3744), (37.33...
6   * UA 33  병천천  [(36.3904, 127.2103), (36.3902, 127.2111), (36...
7   * UA 34  미호천  [(36.371, 127.2048), (36.3705, 127.2105), (36....
8   * UA 35   김해  [(35.2057, 128.4815), (35.2101, 128.4825), (35...
9   * UA 36   밀양  [(35.2801, 128.4642), (35.2729, 128.4714), (35...
10  * UA 37   창원  [(35.2238, 128.3856), (35.2238, 128.3931), (35...
11  * UA 38   울주  [(35.3129, 129.0947), (35.3128, 129.0957), (35...
12  * UA 39   김제  [(35.5435, 126.5304), (35.5454, 126.5257), (35...
13  * UA 41   대전  [(36.2754, 12